In [1]:
import xml.etree.ElementTree as ET
from pathlib import Path
from typing import Union, Tuple, Set, Dict

# yEd / GraphML namespaces
NS = {
    "g": "http://graphml.graphdrawing.org/xmlns",
    "y": "http://www.yworks.com/xml/graphml",
}

def ensure_bidirectional_edges_in_graphml(
    graphml_in: Union[str, Path],
    graphml_out: Union[str, Path],
    *,
    skip_self_loops: bool = True,
    adjust_dx_dy_keys: Tuple[str, str] = ("d_dx", "d_dy"),  # if present on edges, negate for reverse
) -> None:
    """
    Ensure that for every edge (source -> target) there exists a reverse edge (target -> source).
    - Copies ALL <data> blocks from the original edge (style + custom data keys).
    - Keeps node labels and edge styles/colors exactly as-is.
    - Generates unique ids for new edges (e.g., "<old>_rev", "<old>_rev2", ...).
    - If adjust_dx_dy_keys are present among <data key="...">, the values are negated for the reverse edge.

    Parameters
    ----------
    graphml_in : path to input GraphML
    graphml_out: path to write updated GraphML
    skip_self_loops : if True, do not add reverse edges for u == v
    adjust_dx_dy_keys: keys to negate for the reverse edge if found
        (use None/empty to disable special handling)
    """
    tree = ET.parse(str(graphml_in))
    root = tree.getroot()

    # Collect all used edge ids to avoid collisions when creating new ones
    used_ids: Set[str] = set(e.get("id") for e in root.findall(".//g:graph/g:edge", NS) if e.get("id"))

    def unique_edge_id(base: str) -> str:
        """Return a unique id not in used_ids, starting with base."""
        candidate = base
        k = 2
        while candidate in used_ids:
            candidate = f"{base}{k}"
            k += 1
        used_ids.add(candidate)
        return candidate

    # Process EACH <graph> separately (yEd can have nested graphs)
    for graph in root.findall(".//g:graph", NS):
        # Map (s,t) -> one representative edge element
        st_to_edge: Dict[Tuple[str, str], ET.Element] = {}

        for e in graph.findall("./g:edge", NS):
            s = e.get("source")
            t = e.get("target")
            if not s or not t:
                continue
            st_to_edge[(s, t)] = e

        # Determine which reverse edges are missing
        to_add: list[Tuple[ET.Element, str, str]] = []
        for (s, t), e in st_to_edge.items():
            if skip_self_loops and s == t:
                continue
            if (t, s) not in st_to_edge:
                to_add.append((e, t, s))

        # Add reverse edges
        for orig_edge, new_source, new_target in to_add:
            # Make a shallow copy of the edge element (tag + attributes only)
            new_edge = ET.Element(f"{{{NS['g']}}}edge", attrib={
                "id": unique_edge_id((orig_edge.get("id") or "e") + "_rev"),
                "source": new_source,
                "target": new_target,
            })

            # Copy all <data> children (style + custom data)
            for d in orig_edge.findall("./g:data", NS):
                new_d = ET.SubElement(new_edge, f"{{{NS['g']}}}data", attrib=d.attrib.copy())

                # Copy style/content subtree if any (e.g., y:PolyLineEdge / y:GenericEdge)
                for child in list(d):
                    new_d.append(_deepcopy_xml(child))

                # If this <data> is a plain text node (keyed data), copy .text and adjust dx/dy if needed
                if d.text and d.text.strip() != "":
                    txt = d.text
                    key = d.attrib.get("key", "")
                    if adjust_dx_dy_keys and key in adjust_dx_dy_keys:
                        # Try to negate numeric strings safely
                        try:
                            val = float(txt)
                            txt = f"{-val:.6f}"
                        except Exception:
                            pass
                    new_d.text = txt

            # Append new edge to the same graph
            graph.append(new_edge)

    # Register namespaces and write out
    ET.register_namespace("", NS["g"])
    ET.register_namespace("y", NS["y"])
    tree.write(str(graphml_out), encoding="utf-8", xml_declaration=True)


# ---------- helper: deep copy an XML subtree (ElementTree has no builtin deepcopy) ----------
def _deepcopy_xml(elem: ET.Element) -> ET.Element:
    new_elem = ET.Element(elem.tag, attrib=elem.attrib.copy())
    new_elem.text = elem.text
    new_elem.tail = elem.tail
    for child in list(elem):
        new_elem.append(_deepcopy_xml(child))
    return new_elem


In [3]:
ensure_bidirectional_edges_in_graphml(
    graphml_in="../mapGraphs/graphml/HIMCM_graph_final_addWeight.graphml",
    graphml_out="../mapGraphs/graphml/HIMCM_graph_FINAL.graphml",
    skip_self_loops=True,
    adjust_dx_dy_keys=("d_dx", "d_dy")
)
